# Training and testing target-specific machine-learning models for structure-based virtual screening

This Jupyter notebook helps users train and test target-specific classification-based machine-learning models for structure-based virtual screening using PLEC features.

Additional information can be found in our Nature Protocols paper: Tran-Nguyen, V. K., Junaid, M., Simeon, S. & Ballester, P. J. A practical guide to machine-learning scoring for structure-based virtual screening. Nat. Protoc. (2023)

We recommend users to set up the protocol-env environment before running the code in this Jupyter notebook. This can be done using the protocol-env.yml file in our MLSF-protocol github repository: https://github.com/vktrannguyen/MLSF-protocol.

For deepcoys generation, please use the github repository https://github.com/fimrie/DeepCoy/tree/master

## 1. Import all necessary Python packages

The following libraries/packages/toolkits need to be installed beforehand: jupyter notebook, pandas, oddt, sklearn, xgboost, rdkit, deepchem, joblib, tqdm, glob, tensorflow. 

In [ ]:
import os
import numpy as np
import pandas as pd
import oddt
import oddt.pandas as opd
from oddt.pandas import ChemDataFrame
from oddt.fingerprints import PLEC
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from xgboost.sklearn import XGBClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval

## 2. Load smile files for train and test

Examples of these csv data files are provided in our MLSF-protocol github repository: https://github.com/vktrannguyen/MLSF-protocol. Please refer to our Nature Protocols paper cited above for more information.

In [ ]:
#Provide the pathway to the csv training data file:
train_data = pd.read_csv("pathway_to_training-set_csv_data_file")

#Provide the pathway to the csv test data file:
test_data = pd.read_csv("pathway_to_test-set_csv_data_file")

## 3. Convert smiles to mol2 files

In [ ]:
for index,smi in enumerate(chem_file_1_smiles):
    mol=pybel.readstring(string=smi,format='smiles')
    mol.title='mol_'+str(index)
    mol.make3D('mmff94s')
    mol.localopt(forcefield='mmff94s', steps=500)
    out=pybel.Outputfile(filename='/path_for_mol2_files/'+'mol_'+str(index)+'.mol2',format='mol2',overwrite=True)
    out.write(mol)
    out.close()

## 4. Generate 30 conformation for each molecule

In [ ]:
def generate_conformers_and_save_sdf(smiles_list, max_conformers=30):
    for idx, smiles in enumerate(smiles_list):
        mol = pybel.readstring("smi", smiles)
        mol.make3D()

        for conformer_num in range(1, max_conformers + 1):
            mol_copy = mol.clone
            mol_copy.localopt(forcefield="mmff94", steps=500)  # Perform local optimization
            sdf_filename = f"mol_{idx+1}_{conformer_num}.mol2"
            mol_copy.write("mol2", sdf_filename, overwrite=True)
generate_conformers_and_save_sdf(smiles_list, max_conformers=30)

## 5. Run docking using smina

In [ ]:
def run_smina(input_file):
    receptor_path = ['path_to_receptor_pdb_file']
    input_path = ['path_to_mol2_files' + str(input_file)]
    output_path = ['path_for_output_docked_poses/' + str(input_file)[:-5] +  '_docked.sdf']
    smina_command = ['path_to_smina_code/smina -r '+ str(receptor_path[0])+ 
                    ' -l '+str(input_file)+ ' -o '+str(output_path[0])+ ' --center_x ' +str(center['center_x'])+
                    ' --center_y '+str(center['center_y'])+ ' --center_z '+str(center['center_z'])+ 
                    ' --size_x '+ str(size['size_x'])+ ' --size_y '+ str(size['size_y'])+ ' --size_z '+ str(size['size_z'])+ ' --exhaustiveness 8 --num_modes 1']
    os.system(smina_command[0])
    
Parallel(n_jobs = 40, backend = 'multiprocessing')(delayed(run_smina)(input_file) for input_file in tqdm(input_files))

## 6. Extract PLEC fingerprints from input data

In [ ]:
protein = next(oddt.toolkit.readfile('pdb', 'path_to_receptor_structure'))
def parallel_plec(lig):
    ligand = next(oddt.toolkit.readfile('sdf', lig))
    feature = PLEC(ligand, protein = protein, size = 4092, 
                  depth_protein = 4, depth_ligand = 2,
                  distance_cutoff = 4.5, sparse = False)
    return feature
plec_training_actives = Parallel(n_jobs = 40, backend = "multiprocessing")(delayed(parallel_plec)(mol) for mol in tqdm(docked_sdf_active))

## 7. Train and test the RF algorithm

hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"n_estimators": hp.uniform("n_estimators", 50, 10000),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5, None]),
         "criterion": hp.choice("criterion", ['gini', 'entropy']),
         'min_samples_leaf':hp.randint('min_samples_leaf',1,5),
         'min_samples_split':hp.randint('min_samples_split',2,6)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(**space, n_jobs = 40)
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(fn = hyperparameter_tuning_randomforest, space = space, algo = tpe.suggest,
                              max_evals = 10, trials = trials)
best_params = space_eval(space, best_rf_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the RF model on the training molecules, using optimal parameters:
rf_plec = RandomForestClassifier(n_estimators = int(best_params['n_estimators']), 
                                 max_depth = best_params['max_depth'], 
                                 criterion = best_params['criterion'],
                                 min_samples_split = best_params['min_samples_split'],
                                 min_samples_leaf = best_params['min_samples_leaf']
                                 n_jobs = 30)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_plec_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})
plec_result_rf.to_csv("where_you_want_to_store_the_csv_output_file")

## 8. Train and test the XGB algorithm

hyperparameter tuning

In [ ]:
seed=5
def objective_plec_2(params):
    xgb_plec=xgb.XGBClassifier(**params, n_jobs=40, random_state=seed)
    xgb_plec.fit(X_train_plec,y_train_plec)
    pred_xgb_plec=xgb_plec.predict(X_val_plec)
    accuracy = accuracy_score(y_val_plec, pred_xgb_plec)
    return {'loss': -accuracy, 'status': STATUS_OK}

params={'n_estimators':hp.randint('n_estimators',100,500),
           'max_depth':hp.randint('max_depth',5,20),
           'learning_rate':hp.choice('learning_rate',[0.01,0.1])}

def optimize_plec_2(trial_xgb_plec):

    best_plec_2=fmin(fn=objective_plec_2,
                     space=params,
                     algo=tpe.suggest,
                     max_evals=500,
                     rstate=np.random.default_rng(seed))
    return best_plec_2

trial_xgb_plec=Trials()
best_xgb_plec=optimize_plec_2(trial_xgb_plec)

In [ ]:
#Train the XGB model on the training molecules, using optimal parameters:
xgb_plec = XGBClassifier(max_depth = int(best_params['max_depth']),
                         n_estimators = int(best_params['n_estimators']),
                         learning_rate = int(best_params['learning_rate'])
                         n_jobs = 40, random_state = 0)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})
plec_result_xgb.to_csv("where_you_want_to_store_the_csv_output_file")

## 9. Train and test the SVM algorithm

hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"C": hp.uniform("C", 0, 20),
         "gamma": hp.choice("gamma", ['scale', 'auto']),
         "kernel": hp.choice("kernel", ['rbf', 'poly', 'sigmoid'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_SVM(space):
    with parallel_backend(backend = "multiprocessing", n_jobs = 40):
        model = SVC(C = space['C'],
                    gamma = space['gamma'],
                    kernel = space['kernel'])
        model.fit(np.array(train_features), Train_Class)
        predicted_train = model.predict(np.array(train_features))
        mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
        
#Search for optimal parameters:
trials = Trials()
best_svm_classification = fmin(fn = hyperparameter_tuning_SVM, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_svm_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the SVM model on the training molecules, using optimal parameters:
svm_plec = SVC(C = best_params['C'], gamma = best_params['gamma'], kernel = best_params['kernel'], 
               probability = True, random_state = 0)
svm_plec.fit(train_features, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(test_features)
prediction_test_svm_plec_prob = svm_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Active_Prob": prediction_test_svm_plec_prob[:, 0],
                                 "Inactive_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})
plec_result_svm.to_csv("where_you_want_to_store_the_csv_output_file")

## 10. Train and test the ANN algorithm

hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"hidden_layer_sizes": hp.uniform("hidden_layer_sizes", 8, 140),
         "activation": hp.choice("activation", ['relu', 'tanh']),
         "max_iter": hp.uniform("max_iter", 1000, 10000)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_ANN(space):
    model = MLPClassifier(hidden_layer_sizes = int(space['hidden_layer_sizes']),
                          activation = space['activation'],
                          max_iter = int(space['max_iter']))
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_ann_classification = fmin(fn = hyperparameter_tuning_ANN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_ann_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the ANN model on the training molecules, using optimal parameters:
ann_plec = MLPClassifier(hidden_layer_sizes = int(best_params['hidden_layer_sizes']), 
                         activation = best_params['activation'], 
                         max_iter = int(best_params['max_iter']), 
                         random_state = 0)
ann_plec.fit(train_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})
plec_result_ann.to_csv("where_you_want_to_store_the_csv_output_file")